<a href="https://colab.research.google.com/github/nytrp/sistemas-inteligentes-para-electronica/blob/main/Exposici%C3%B3n%20-%20Corte%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **🌐Redes Neuronales Recurrentes (RNN)**

## 👥 Grupo:
### Alejandro Martinez, Yurliana Niebles, Isaac Montes, Daniel Santiago

---

## 🧠 ¿Qué es una Red Neuronal Recurrente?

Una **red neuronal recurrente (RNN)** es un tipo de red neuronal profunda diseñada para procesar datos secuenciales como series de tiempo, texto, voz o secuencias de eventos, generando salidas que también respetan ese orden.

A diferencia de una red neuronal tradicional (feed-forward), que asume que cada entrada es independiente, una RNN mantiene información de estados anteriores y la utiliza para influir en las predicciones actuales. Esto le permite modelar relaciones de contexto temporal.

---

## ⚙️ ¿Cómo Funciona?

La representación de una RNN muestra cómo la misma neurona se aplica en distintos instantes de tiempo $ t $, recibiendo la entrada $ x_t $ y produciendo una salida $ y_t $. El estado oculto $ a_{t-1} $ se retroalimenta al siguiente paso, actuando como memoria del contexto previo y permitiendo que la red recuerde información anterior para influir en la salida presente.

- **Características Clave:**
  - Todas las neuronas comparten los mismos pesos, lo que permite procesar secuencias de distinta longitud sin modificar la arquitectura.
  - El entrenamiento se realiza mediante **retropropagación a través del tiempo (BPTT)**, ajustando los parámetros en cada paso para que la información relevante fluya a través del estado oculto y mejore las predicciones.

En resumen, una RNN es capaz de capturar dependencias temporales en los datos gracias a su memoria interna.

---

## 🎯 ¿Para Qué Sirve?

Las RNN tienen aplicaciones en cualquier dominio donde los datos sean secuenciales y el orden sea importante. Se utilizan ampliamente en:

- **Procesamiento de Lenguaje Natural:** Traducción automática, generación de texto, análisis de sentimientos.
- **Reconocimiento de Voz y Audio:** Transcripción y respuesta manteniendo en memoria lo ya escuchado.
- **Predicción de Series Temporales:** Precios financieros, pronóstico del clima, detección de anomalías en sensores.
- **Visión Secuencial:** Combinación con redes convolucionales para subtitular imágenes o videos y describir escenas de forma coherente.

En resumen, las RNN destacan en tareas donde se debe modelar una secuencia de eventos o señales, ya que su capacidad de recordar el contexto previo les permite tomar decisiones más precisas en aplicaciones del mundo real.

---

## 📊 Ventajas y Desventajas

Las RNN son muy potentes para trabajar con datos secuenciales porque pueden recordar información previa y adaptarse a secuencias de cualquier longitud. Sin embargo, su entrenamiento suele ser lento y complejo, y pueden tener problemas para aprender dependencias muy largas.

### **✅ Ventajas:**
- Memoria a corto y largo plazo (con LSTM o GRU).
- Manejan secuencias de distinta longitud sin cambiar la arquitectura.
- Buenas para datos donde el orden importa (texto, voz, series temporales).
- Pueden combinarse con otras redes como CNN para tareas más complejas.

### **❌ Desventajas:**
- Problemas de gradiente (explosivo o desvaneciente).
- Entrenamiento lento y costoso por su naturaleza secuencial.
- Dificultad para capturar dependencias muy largas.
- Más complicadas de ajustar e interpretar que modelos clásicos.
- Menos eficientes que arquitecturas modernas como los Transformers.

---


In [1]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install tensorflow


In [ ]:
# =======================================
# 1. Subir archivo desde tu PC
# =======================================
from google.colab import files
uploaded = files.upload()   # Selecciona electricity.csv

# =======================================
# 2. Importar librerías
# =======================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# =======================================
# 3. Cargar dataset
# =======================================
df = pd.read_csv("electricity.csv")
print(df.head())
print(df.shape)

# Usar última columna como objetivo (ajusta si es necesario)
target_col = df.columns[-1]
print("Usando la columna objetivo:", target_col)

# =======================================
# 4. Normalización
# =======================================
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df[[target_col]])

# =======================================
# 5. Crear ventanas de secuencia
# =======================================
def create_sequences(data, seq_length=48):  # ventana de 48 pasos
    X, y = [], []
    for i in range(len(data)-seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 48
X, y = create_sequences(scaled, seq_length)

# División en train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

print("Shapes ->", X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# =======================================
# 6. Modelo RNN con múltiples capas LSTM
# =======================================
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(seq_length, 1)),
    Dropout(0.2),
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
print(model.summary())

# =======================================
# 7. Entrenamiento del modelo
# =======================================
history = model.fit(
    X_train, y_train,
    epochs=20, batch_size=64,  # más épocas
    validation_split=0.2
)

# =======================================
# 8. Evaluación del modelo RNN
# =======================================
preds = model.predict(X_test)
mse = mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
print(f"RNN - MSE: {mse:.4f}, R²: {r2:.4f}")

# =======================================
# 9. Comparación con Regresión Lineal
# =======================================
X_train_lr = X_train.reshape(X_train.shape[0], -1)
X_test_lr  = X_test.reshape(X_test.shape[0], -1)

lr = LinearRegression()
lr.fit(X_train_lr, y_train)
lr_preds = lr.predict(X_test_lr)

mse_lr = mean_squared_error(y_test, lr_preds)
r2_lr = r2_score(y_test, lr_preds)
print(f"Regresión Lineal - MSE: {mse_lr:.4f}, R²: {r2_lr:.4f}")

# =======================================
# 10. Visualización
# =======================================
plt.figure(figsize=(12,5))
plt.plot(scaler.inverse_transform(y_test.reshape(-1,1)), label="Real")
plt.plot(scaler.inverse_transform(preds), label="Predicción RNN (LSTM, múltiples capas)")
plt.plot(scaler.inverse_transform(lr_preds.reshape(-1,1)), label="Predicción Regresión Lineal", alpha=0.7)
plt.legend()
plt.title("Predicción de Serie Temporal (electricity.csv)")
plt.show()